In [ ]:
!pip install bing-image-downloader

In [ ]:
import tensorflow as tf
import numpy as np
from bing_image_downloader import downloader

In [24]:
CLASSES = [
	'macbook',
	'toilet',
	'equation',
	'programming',
	'keyboard',
	'phone',
]
IMGS_PER_CLASS = 20

In [25]:
# Download data
for class_name in CLASSES:
	response = google_images_download.googleimagesdownload()
	downloader.download(
		class_name,
		limit=IMGS_PER_CLASS,
		output_dir=class_name,
		adult_filter_off=False,
		force_replace=False,
		timeout=60,
		verbose=False
	)

[%] Downloading Images to /home/jankaifer/Projects/acap-ocr/tensorflow/macbook/macbook
[!] Issue getting: https://appleinclusion.com/wp-content/uploads/2019/05/howard-lawrence-b-1141813-unsplash-1.jpg
[!] Error:: HTTP Error 404: Not Found
[!] Issue getting: https://external-preview.redd.it/5BCTe6GcIWnfl0RT6W9jsNIOeRoQ4K36oc24vpmzAeY.jpg?auto=webp&amp;s=2c9d35b3a5b39c78d0699d35c48e34a6214d2cb3
[!] Error:: HTTP Error 403: Forbidden
[!] Issue getting: https://preview.redd.it/b0oy5ewj05e51.jpg?auto=webp&amp;s=be3cf283476f60f65b4b1d058c0441ddc2c93a21
[!] Error:: HTTP Error 403: Forbidden


KeyboardInterrupt: 